In [65]:
import numpy as np 
import os
import matplotlib.pyplot as plt
from matplotlib import transforms
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patheffects as PathEffects

from astropy.io import fits 
from astropy.convolution import convolve, Gaussian2DKernel
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.table import Table
from astropy.visualization import ImageNormalize, ZScaleInterval, LinearStretch, MinMaxInterval, LogStretch

In [2]:
#Set the cosmology to use and the redshift. 
cosmo = FlatLambdaCDM(H0=70., Om0=0.3)
z = 2.099

In [3]:
base_fname = "../../Calculations/SKIRTOR/2D_maps/bHDPol_mrn77_tor_oa60_con_oa30-tauV1_i75"
I = fits.open(base_fname+"_total.fits")
Q = fits.open(base_fname+"_stokesQ.fits")
U = fits.open(base_fname+"_stokesU.fits")

In [ ]:
P = (Q[0].data**2+U[0].data**2)**0.5/I[0].data
Chi = 0.5*np.arctan2(U[0].data, Q[0].data)*180./np.pi

In [5]:
#Get the model file to read the wavelength. 
wave = np.loadtxt("../../Calculations/SKIRTOR/models/bHDPol_mrn77_tor_oa60_con_oa30-tauV1_i75_sed.dat", usecols=[0])
wave = wave * u.micron

In [ ]:
#Find the closest the center of the R-band effective wavelength of 6500. 
wave_targ_FORS2 = 6500*u.AA / (1+z)
k_FORS2 = np.argmin(np.abs(wave-wave_targ_FORS2))

#Also the closest to the F160W value
wave_targ_F555W = 5550*u.AA / (1+z)
k_F555W = np.argmin(np.abs(wave-wave_targ_F555W))

#Also the closest to the F160W value
wave_targ_F160W = 1.6*u.micron / (1+z)
k_F160W = np.argmin(np.abs(wave-wave_targ_F160W))

print(k_FORS2, k_F555W, k_F160W)
print(wave[k_FORS2]*(1+z), wave[k_F555W]*(1+z), wave[k_F160W]*(1+z))

In [7]:
#Find the worse seeing of the FORS2 observations. 
all_seeing = Table.read("../../Resolved_Polarization_v3/All_seeing_values.txt", format='ascii')
common_seeing = np.max(all_seeing['col3'][all_seeing['col1']=="W0204-0506"])
common_seeing = common_seeing * u.arcsec

In [ ]:
#To match the observed size, the image would roughly have to be 20kpc x 20kpc. From there we can figure out the pixel scale and match the PSF sizes. 
ny, nx = I[0].data[k_FORS2].shape 
phys_size_sim = 20*u.kpc
ang_size_sim = (phys_size_sim/cosmo.angular_diameter_distance(z)).to(u.dimensionless_unscaled)*u.rad
pix_scale_sim = ang_size_sim/nx
print(ang_size_sim.to(u.arcsec), pix_scale_sim.to(u.arcsec))

In [ ]:
#Factor for converting between FWHM and sigma. Needed since the Gaussian2DKernel takes dispersions. 
FWHM_to_sigma_factor = 2*(2*np.log(2))**0.5
print(FWHM_to_sigma_factor)

In [10]:
#Convolve with the FORS2 PSF. First, try to read the images. If they do not exist, run the convolution. 
if os.path.exists("I_FORS2.fits"):
    I_FORS2 = fits.getdata("I_FORS2.fits")
    Q_FORS2 = fits.getdata("Q_FORS2.fits")
    U_FORS2 = fits.getdata("U_FORS2.fits")
else:
    #Set the convolution kernel. 
    gauss_kernel = Gaussian2DKernel(int(common_seeing/FWHM_to_sigma_factor/pix_scale_sim))#, x_size=ny, y_size=nx) 

    #Replace the nans with 0 to make convolution easier. 
    I_use = np.where(np.isnan(I[0].data[k_FORS2]), 0., I[0].data[k_FORS2])
    Q_use = np.where(np.isnan(Q[0].data[k_FORS2]), 0., Q[0].data[k_FORS2])
    U_use = np.where(np.isnan(U[0].data[k_FORS2]), 0., U[0].data[k_FORS2])

    #Run convolution 
    I_FORS2 = convolve(I_use, gauss_kernel)
    Q_FORS2 = convolve(Q_use, gauss_kernel)
    U_FORS2 = convolve(U_use, gauss_kernel)

    #Save the convolved images.
    fits.writeto("I_FORS2.fits", I_FORS2)
    fits.writeto("Q_FORS2.fits", Q_FORS2)
    fits.writeto("U_FORS2.fits", U_FORS2)

In [11]:
#Get the polarization fraction and angle files. 
P_FORS2 = (Q_FORS2**2+U_FORS2**2)**0.5 / I_FORS2
Chi_FORS2 = 0.5*np.arctan2(U_FORS2,Q_FORS2)*180./np.pi


In [12]:
#Convolve with the F555W PSF. 
F555W_FWHM = 0.067*u.arcsec
if os.path.exists("I_F555W.fits"):
    I_F555W = fits.getdata("I_F555W.fits")
    Q_F555W = fits.getdata("Q_F555W.fits")
    U_F555W = fits.getdata("U_F555W.fits")
else:
    #Set the convolution kernel. 
    gauss_kernel = Gaussian2DKernel(int(F555W_FWHM/FWHM_to_sigma_factor/pix_scale_sim)) 

    #Replace the nans with 0 to make convolution easier. 
    I_use = np.where(np.isnan(I[0].data[k_F555W]), 0., I[0].data[k_F555W])
    Q_use = np.where(np.isnan(Q[0].data[k_F555W]), 0., Q[0].data[k_F555W])
    U_use = np.where(np.isnan(U[0].data[k_F555W]), 0., U[0].data[k_F555W])

    #Convolve the images. 
    I_F555W = convolve(I_use, gauss_kernel)
    Q_F555W = convolve(Q_use, gauss_kernel)
    U_F555W = convolve(U_use, gauss_kernel)

    #Save the convolved images.
    fits.writeto("I_F555W.fits", I_F555W)
    fits.writeto("Q_F555W.fits", Q_F555W)
    fits.writeto("U_F555W.fits", U_F555W)

In [ ]:
#Get the polarization fraction and angle files. 
P_F555W = (Q_F555W**2+U_F555W**2)**0.5 / I_F555W
Chi_F555W = 0.5*np.arctan2(U_F555W,Q_F555W)*180./np.pi

In [18]:
#Convolve with the F160W PSF. 
F160W_FWHM = 0.148*u.arcsec
if os.path.exists("I_F160W.fits"):
    I_F160W = fits.getdata("I_F160W.fits")
    Q_F160W = fits.getdata("Q_F160W.fits")
    U_F160W = fits.getdata("U_F160W.fits")
else:
    #Set the convolution kernel. 
    gauss_kernel = Gaussian2DKernel(int(F160W_FWHM/FWHM_to_sigma_factor/pix_scale_sim)) 

    #Replace the nans with 0 to make convolution easier. 
    I_use = np.where(np.isnan(I[0].data[k_F160W]), 0., I[0].data[k_F160W])
    Q_use = np.where(np.isnan(Q[0].data[k_F160W]), 0., Q[0].data[k_F160W])
    U_use = np.where(np.isnan(U[0].data[k_F160W]), 0., U[0].data[k_F160W])

    #Convolve the images. 
    I_F160W = convolve(I_use, gauss_kernel)
    Q_F160W = convolve(Q_use, gauss_kernel)
    U_F160W = convolve(U_use, gauss_kernel)

    #Save the convolved images.
    fits.writeto("I_F160W.fits", I_F160W)
    fits.writeto("Q_F160W.fits", Q_F160W)
    fits.writeto("U_F160W.fits", U_F160W)

In [ ]:
#Get the polarization fraction and angle files. 
P_F160W = (Q_F160W**2+U_F160W**2)**0.5 / I_F160W
Chi_F160W = 0.5*np.arctan2(U_F160W,Q_F160W)*180./np.pi

In [ ]:
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(20,20))
for ax in axs.flatten():
    ax.set_aspect('equal')
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])
plt.subplots_adjust(wspace=0.0,hspace=0.0)

#For FORS2, only use the places where the ratio between the max and the min is about 0.17, the same as is in the real pol image shown in the paper. 
I_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), I_FORS2, np.nan)
P_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), P_FORS2, np.nan)
Chi_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), Chi_FORS2, np.nan)

norm = ImageNormalize(I[0].data[k_FORS2], stretch=LinearStretch(), interval=ZScaleInterval())
print(norm.__dict__)
for k, Iim in enumerate([I[0].data[k_FORS2], I_FORS2_use, I_F555W]):
    tr = transforms.Affine2D().rotate_deg_around(Iim.shape[1]/2., Iim.shape[0]/2., -70)
    cb = axs[0,k].imshow(Iim, origin='lower', transform=tr + axs[0,k].transData, cmap='gray_r', vmin=5.e-13, vmax=5e-11)#, norm=norm)
    cbaxes = inset_axes(axs[0,k], width="5%", height="95%",loc='center left')
    fig.colorbar(cb, cax=cbaxes, orientation='vertical')

Pims = [P[k_FORS2], P_FORS2_use, P_F555W]
for k, Pim in enumerate(Pims):
    tr = transforms.Affine2D().rotate_deg_around(Pim.shape[1]/2., Pim.shape[0]/2., -70 + 180)
    Pim_use = np.where((Pim<=1.0) & (Pim>=0.0), Pim, np.nan)
    cb = axs[1,k].imshow(Pim_use, origin='lower', transform=tr + axs[1,k].transData, cmap='plasma_r')#, vmin=0., vmax=1.)
    cbaxes = inset_axes(axs[1,k], width="5%", height="95%",loc='center left')
    fig.colorbar(cb, cax=cbaxes, orientation='vertical')

for k, Cim in enumerate([Chi[k_FORS2], Chi_FORS2_use, Chi_F555W]):
    tr = transforms.Affine2D().rotate_deg_around(Cim.shape[1]/2., Pim.shape[0]/2., -70 + 180)
    Cim_use = np.where((Pims[k]<=1.0) & (Pims[k]>=0.0), Cim, np.nan)
    Cim_use[Cim_use<0.] += 180.
    cb = axs[2,k].imshow(Cim_use, origin='lower', transform=tr + axs[2,k].transData, vmin=0., vmax=180., cmap='hsv')
    cbaxes = inset_axes(axs[2,k], width="5%", height="95%",loc='center left')
    fig.colorbar(cb, cax=cbaxes, orientation='vertical')


plt.show()

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(40,9), squeeze=False)
plt.subplots_adjust(wspace=0., hspace=0.)
for ax in axs.flat:
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim([0,700])

tr = transforms.Affine2D().rotate_deg_around(I_FORS2_use.shape[1]/2., I_FORS2_use.shape[0]/2., -70)

#For FORS2, only use the places where the ratio between the max and the min is about 0.17, the same as is in the real pol image shown in the paper. 
I_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), I_FORS2, np.nan)
P_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), P_FORS2, np.nan)
Chi_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), Chi_FORS2, np.nan)
Chi_FORS2_use[Chi_FORS2_use<0.] += 180.
Chi_FORS2_use -= 70
Chi_FORS2_use[Chi_FORS2_use<0.] += 180.

#First panel shows the pol angle in FORS2.
cb = axs[0,0].imshow(Chi_FORS2_use, transform=tr+axs[0,0].transData, origin='lower', cmap='hsv', vmin=0., vmax=180.)
cbaxes = inset_axes(axs[0,0], width="5%", height="95%",loc='center right')
fig.colorbar(cb, cax=cbaxes, orientation='vertical')

#Second shows the polarization fraction. 
cb = axs[0,1].imshow(P_FORS2_use, transform=tr+axs[0,1].transData, origin='lower', cmap='plasma_r')#, vmin=0., vmax=180.)
cbaxes = inset_axes(axs[0,1], width="5%", height="95%",loc='center right')
fig.colorbar(cb, cax=cbaxes, orientation='vertical')

#Third shows the F555W intensity. 
norm = ImageNormalize(I_F555W, stretch=LogStretch(), interval=MinMaxInterval())
cb = axs[0,2].imshow(I_F555W, transform=tr+axs[0,2].transData, origin='lower', cmap='gray_r', norm=norm)
#cbaxes = inset_axes(axs[0,2], width="5%", height="95%",loc='center left')
#fig.colorbar(cb, cax=cbaxes, orientation='vertical')
axs[0,2].set_facecolor('white')

#Fourth shows the F160W intensity. 
norm = ImageNormalize(I_F160W, stretch=LogStretch(), interval=MinMaxInterval())
cb = axs[0,3].imshow(I_F160W, transform=tr+axs[0,3].transData, origin='lower', cmap='gray_r', norm=norm)
#cbaxes = inset_axes(axs[0,3], width="5%", height="95%",loc='center left')
#fig.colorbar(cb, cax=cbaxes, orientation='vertical')
axs[0,3].set_facecolor('white')

for ax in axs.flat:
    print(ax.get_xlim())


In [ ]:
#Now, plot the panels. 
fig, axs = plt.subplots(1,4, figsize=(40,9), squeeze=False)
plt.subplots_adjust(wspace=0., hspace=0.)
for ax in axs.flat:
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xlim([0,700])


tr = transforms.Affine2D().rotate_deg_around(I_FORS2_use.shape[1]/2., I_FORS2_use.shape[0]/2., -70+180)

#For FORS2, only use the places where the ratio between the max and the min is about 0.17, the same as is in the real pol image shown in the paper. 
I_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), I_FORS2, np.nan)
P_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), P_FORS2, np.nan)
Chi_FORS2_use = np.where((P_FORS2<1.0) & (I_FORS2>0.17*np.max(I_FORS2)), Chi_FORS2, np.nan)
Chi_FORS2_use[Chi_FORS2_use<0.] += 180.
Chi_FORS2_use -= 70
Chi_FORS2_use[Chi_FORS2_use<0.] += 180.

#Plot the images.
cm_pa = axs[0,0].imshow(Chi_FORS2_use, transform=tr+axs[0,0].transData, vmin=0., vmax=180., cmap='hsv', origin='lower', aspect='auto')

cm_pf = axs[0,1].imshow(P_FORS2_use, transform=tr+axs[0,1].transData, cmap='plasma_r', origin='lower', aspect='auto')#, vmin=15., vmax=35.

norm_f555w = ImageNormalize(I_F555W, stretch=LogStretch(), interval=MinMaxInterval())#, vmin=-0.05, vmax=0.08)
axs[0,2].imshow(I_F555W, transform=tr+axs[0,2].transData, norm=norm_f555w, cmap='gray_r', origin='lower', aspect='auto')

norm_f160w = ImageNormalize(I_F160W, stretch=LogStretch(), interval=MinMaxInterval())#, vmin=-1.2, vmax=2.0)
axs[0,3].imshow(I_F160W, transform=tr+axs[0,3].transData, norm=norm_f160w, cmap='gray_r', origin='lower', aspect='auto')

#Add the colorbars to the polarization panels.
for k, cm in enumerate([cm_pa, cm_pf]):
    if k==0:
        label = r'Polarization Angle $\chi~(\rm deg)$'
    else:
        label = r'Polarization Fraction $P~(\%)$'
    cax = inset_axes(axs[0,k], width="5%", height="90%", loc='center right') 
    cbar = fig.colorbar(cm, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=18)
    cbar.set_label(label=label, fontsize=24)
    cax.yaxis.set_ticks_position('left')
    cax.yaxis.set_label_position('left')

#Draw an x at the position of the AGN.
for ax in axs.flatten():
    ax.scatter([300],[300],color='white', marker='x', s=500, lw=2.0, path_effects=[PathEffects.withStroke(linewidth=10, foreground='black')])
    #sc.set_path_effects([PathEffects.withStroke(linewidth=6, foreground='black')])

#Add the bands.
for k, bandname in enumerate([r'$R_{\rm Special}$', r'$R_{\rm Special}$', r'F555W', r'F160W']):
    # if k<2:
    #    color = 'black'
    # else:
    #    color = 'white'
    color = 'black'
    #if k==0:
    #    x, y = 0.05, 0.83
    #else:
        #x, y = 0.05, 0.9
    x, y = 0.03, 0.92
    fontsize=40
    if k<2:
        fontsize=44
    txt = axs[0,k].text(x, y, bandname, transform=axs[0,k].transAxes, fontsize=fontsize, color=color)
    # if k>=2:
    #     txt.set_path_effects([PathEffects.withStroke(linewidth=6, foreground='black')])

#Add a label to show the HST panels show total flux. 
axs[0,0].text(0.03, 0.85, 'Polarization Angle', transform=axs[0,0].transAxes, fontsize=28, color='black')
axs[0,1].text(0.03, 0.85, 'Polarization Fraction', transform=axs[0,1].transAxes, fontsize=28, color='black')
for k in range(2,4):
    txt = axs[0,k].text(0.05, 0.83, 'Total Flux', transform=axs[0,k].transAxes, fontsize=28)#, weight='bold', color='white')
    #txt.set_path_effects([PathEffects.withStroke(linewidth=6, foreground='black')])

#plt.tight_layout()
plt.show()

fig.savefig("W0204_model_2D.png", dpi=200, bbox_inches='tight')